# Optimization

## 1- Mini-Batch Gradient Descent

First I would like to introduce a little the types of Mini-batch sizes. Broadly speaking, the Gradient Descent algorithm is used to find the optimal values ​​of a model's parameters (such as the weights of a neural network) that minimize a cost or loss function. There are three main variants of Gradient Descent:

* **Batch Gradient Descend**: would be the 'default' where calculates the gradient of the cost function using the entire training data set (mini-batch size = m). Accurate, but slow in learning time and needs resources.

* **Stochastic Gradient Descent (SGD)**: when the mini-batch size is 1, calculates the gradient using only one training example at a time. Although it is much faster, it can be noisy and does not always converge stably.

And finally the one used:
* **Mini-Batch Gradient Descent**: It is a combination of the previous two. Instead of using the entire data set or a single example, the algorithm uses small subsets of data called mini-batches. The size of the mini-batch is an *hyperparameter*.

Therefore, we take our traning set and create subsets of the training examples with a *mini-batch size*, then we will update the parameters for each subset.

In [ ]:
""" Mini-Batch Gradient Descent  (optimization 1)"""  
import math

"""
    X --> (n_x,m)   : n_x pixels (features)
    Y --> (n_y,m)   : n_y output_clases
"""
def random_mini_batches(X, Y, mini_batch_size=32):

    # Retrieves the number of total images (training examples)
    m = X.shape[1]
    mini_batches = []

    #Step 1: We create the suffled version for training
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation]

    #Step 2: From shuffled version, we create the minibatches
    inc = mini_batch_size
    num_complete_minibatch = math.floor(m / mini_batch_size)        # Max number of complete batches possible

    for k in range(0,num_complete_minibatch):

        # For each mini-batch k, slices a subset
        mini_batch_X = shuffled_X[:, k*inc : (k+1)*inc]
        mini_batch_Y = shuffled_Y[:, k*inc : (k+1)*inc]

        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    if m % mini_batch_size != 0:
        
        #If not perfectly divisible, last mini-batch with the remaining
        mini_batch_X = shuffled_X[:, num_complete_minibatch*inc : ]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatch*inc : ]

        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    return mini_batches

## 2 - Momentum with Gradient Descent

Momentum is an optimization technique that is used together with the Gradient Descent algorithm to accelerate convergence and overcome problems such as oscillations in the direction of descent. 
* The idea is that instead of simply updating the model parameters using the gradient of the cost function at each step, Momentum adds a fraction of the previous update to the new update. This helps smooth the path to the minimum.

Momentum Formula:  
$$
   v_t = \gamma v_{t-1} + \eta \nabla J(\theta_t)
$$

   Where:
   - \(v_t\) at time 𝑡. It represents the exponentially moving average of the gradients up to time 𝑡.
   - $\gamma$ Momentum coefficient (often denoted as **beta_1** in the context of the Adam optimizer). It controls the contribution of past gradients to the current update.
   - $\eta$ learning rate, an *hyperparameter*.
   - \($\nabla$ J($\theta_t$\)  gradient of the cost function with respect to the parameters at time \(t\).


Now there's another hyperparameter Momentum coefficient  $\eta$, **usually a value of 0.9 or close to it.**

In [ ]:
""" Momentum for Gradient Descent (optimization 2)"""
# When using minibatches for update we make the update just with a subset of examples. 
# Momentum takes in account past gradients to smooth the update.

#Step 1 - Intialize velocity 'v' to zeros (same shape as parameters matrix)
def initilize_velocity(parameters):
    
    L = len(parameters) // 2

    v = {}

    for l in range(1,L+1):

        v["dW"+str(l)] = np.zeros((parameters["W"+str(l)].shape))
        v["db"+str(l)] = np.zeros((parameters["b"+str(l)].shape))

    return v

#Step 2 - Update parameters with momentum
def update_parameters_with_momentum(parameters, grads, v, beta, learning_rate):

    L = len(parameters) // 2

    for l in range(1,L+1):

        #Exponentially Weighted Averages (Formula above)
        v["dW"+str(l)] = beta * v["dW"+str(l)] + (1-beta) * grads["dW"+str(l)]
        v["db"+str(l)] = beta * v["db"+str(l)] + (1-beta) * grads["db"+str(l)]

        #Update of parameters with momentum
        parameters["W"+str(l)] = parameters["W"+str(l)] - learning_rate * v["dW"+str(l)]
        parameters["b"+str(l)] = parameters["b"+str(l)] - learning_rate * v["db"+str(l)]

    return parameters, v

## 3 - Adam Algorithm --> Momentum + RMSProp

The Adam (Adaptive Moment Estimation) optimization technique is one of the most popular and effective algorithms in the field of deep learning. It combines ideas from two optimization techniques: Momentum and RMSprop. Adam adapts the learning rate of each parameter and makes adjustments based on estimates of the first and second moments of the gradient.

* How does Adam Works?

    - First, it calculates an exponentially weighted average of past gradient, and stores it variables before bias correction and after bias correction. (Similar such as *Momentum*)

    - Secondly, it calculates an exponentially weighted average of the squares of the past gradients, and stores it before and after bias correction. (Similar such as *RMSProp*)
    *RMSPROP  maintains a moving average of the squares of the gradients*

    - Finally, it updates the parameters in a direction based on a combining information of the First and Second steps.

* Steps:
### 1. Parameter Initialization
- ( t = 0 )     -->     (iteration/epoch counter).
- ( m_0 = 0 )   -->     (first moment: average of the gradients).
- ( v_0 = 0 )   -->     (second moment: mean of the squares of the gradients).
- ( $beta_1$ and $beta_2$ ) --> (exponential decay coefficients for the first and second moments, respectively).
- ( $alpha$ )   -->     (learning rate).
- ( $epsilon$ ) -->     (numerical stability term, typically ( $10^{-8}$ )).

### 2. Updating the Counter
Increase the iteration counter: $t = t + 1$

### 3. Gradient Calculation
Calculate the cost gradient with respect to the parameters \($\theta$\). In step (t):     $ g_t = \nabla_{\theta} J(\theta_t)$

where \( J($\theta_t$) \) its the cost function.

### 4. First Moment Calculation (Gradient Average)
Update the first moment (average of the gradients):
$$
m_t = \beta_1 \cdot m_{t-1} + (1 - \beta_1) \cdot g_t
$$

### 5. Calculation of the Second Moment (Average of the Squares of the Gradients)
Update the second moment (mean of the squares of the gradients):
$$
v_t = \beta_2 \cdot v_{t-1} + (1 - \beta_2) \cdot g_t^2
$$

### 6. Bias Correction
Correct the bias of the first moment:
$$
\hat{m}_t = \frac{m_t}{1 - \beta_1^t}
$$
Correct the bias of the second moment:
$$
\hat{v}_t = \frac{v_t}{1 - \beta_2^t}
$$

### 7. Update Parameters
Update parameters \( $\theta$ \):
$$
\theta_t = \theta_{t-1} - \alpha \cdot \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}
$$

This process repeats until the model converges.

In [ ]:
""" Adam Algorithm = Momentum + RMSProp (optimization 3)"""
#Step 1: Initialize with Adam:
# 
# matrix to zeros and same shape as parameters
#  
# v --> exponentially weighted average. 
# s --> exponentially weighted average of the squares.
def initialize_parameters_adam(parameters):

    L = len(parameters) // 2

    v = {}
    s = {}

    for l in range(1,L+1):

        v["dW" + str(l)] = np.zeros((parameters["W"+str(l)].shape))
        v["db" + str(l)] = np.zeros((parameters["b"+str(l)].shape))
        s["dW" + str(l)] = np.zeros((parameters["W"+str(l)].shape))
        s["db" + str(l)] = np.zeros((parameters["b"+str(l)].shape))
        
    return v, s

#Step 2: Calculates the respective exponentially weighted average of past gradient before and after bias correction and updates the parameters

def update_parameters_with_adam(parameters, grads, v, s, t, beta1, beta2, learning_rate, epsilon):
    
    L = len(parameters) // 2

    v_corrected = {}
    s_corrected = {}

    for l in range(1, L+1):

        # exponential weighted average 'v' --> formula (4)
        v["dW" + str(l)] = beta1 * v["dW" + str(l)] + (1-beta1) * grads["dW"+str(l)]
        v["db" + str(l)] = beta1 * v["db" + str(l)] + (1-beta1) * grads["db"+str(l)]

        # bias correction 'v' --> formula (6.1)
        v_corrected["dW"+str(l)] = v["dW"+str(l)] / (1-np.power(beta1,t))
        v_corrected["db"+str(l)] = v["db"+str(l)] / (1-np.power(beta1,t))

        # exponentially weighted average of the squares 's'. --> formula (5)
        s["dW"+str(l)] = beta2 * s["dW"+str(l)] + (1 - beta2) * np.power(grads["dW"+str(l)], 2)
        s["db"+str(l)] = beta2 * s["db"+str(l)] + (1 - beta2) * np.power(grads["db"+str(l)], 2)
        
        # bias correction 's' --> formula (6.2)
        s_corrected["dW" + str(l)] = s["dW"+str(l)] / (1-np.power(beta2,t))
        s_corrected["db" + str(l)] = s["db"+str(l)] / (1-np.power(beta2,t))

        # update parameters --> formula (7)
        parameters["W"+str(l)] = parameters["W"+str(l)] - learning_rate * (v_corrected["dW"+str(l)] / (np.sqrt(s_corrected["dW"+str(l)]) + epsilon))
        parameters["b"+str(l)] = parameters["b"+str(l)] - learning_rate * (v_corrected["db"+str(l)] / (np.sqrt(s_corrected["db"+str(l)]) + epsilon))

    return parameters, v, s

## 4 - Regularization to reduce Overfitting

## 4.1 - L2 Regularization

L2 regularization is a technique used to prevent overfitting in models and its goal is to improve the generalization of the model by penalizing large values ​​of model parameters. 

This helps keep the model weights smaller and therefore prevents the model from overfitting the training data.

* The idea is to add a penalty to the term of the model loss function based on the magnitude of the model weights. Mathematically, this is achieved by adding a penalty function to the original loss function.

Function: 
$$
J_{\text{regularized}}(\theta) = J(\theta) + \frac{\lambda}{2} \sum_{i=1}^{n} \theta_i^2
$$

Now, λ is the regularization parameter (also known as regularization coefficient). Control the strength of the penalty. A value of larger λ means a stronger penalty.

The $\sum_{i=1}^{n} \theta_i^2$ is the sum of the squares of all model parameters.

In [ ]:
""" Regularization -- Reduce Overfitting --> L2"""

#Step 1: compute the cost
def compute_cost_with_regularization(AL, Y, parameters, lambd):

    m = Y.shape[1]

    cross_entropy_cost = bce_compute(AL,Y)

    L2_regularization_cost = 0

    for key in parameters:
        if 'W' in key:
            L2_regularization_cost += np.sum(np.square(parameters[key]))


    # we add this formula of above
    L2_regularization_cost = (lambd / (2 * m)) * L2_regularization_cost

    cost = cross_entropy_cost + L2_regularization_cost

    return  cost

## 4.2 - Dropout

Dropout involves randomly deactivating a fraction of the neurons during training of the neural network. This means that, in each training iteration, certain neurons and their connections are temporarily "removed", and do not contribute to the activation or gradient calculation in that iteration.

Visual example: 
<center>
<video width="400" height="260" src="explanations_utils/dropout1_kiank.mp4" type="video/mp4" controls>
</video>
</center>

*Finally, if we are **predicting** we **dont use the dropout***.

In [ ]:
#Paso 3: Forward propagation model with dropout
#
# Now we recieve the dropout probabiliy (a new hyperparameter)
# And layers drop, indicates which layers apply dropout (1 or 0), 1 will be aplied

def L_model_forward(X, parameters, dropout_prob=0, layers_drop=[0, 0, 0, 0, 0]):

    caches = []

    L = len(parameters) // 2
    A = X

    keep_prob = 1 - dropout_prob    # probability of keeping neurons active (1 - dropout_prob)
    
    for l in range(1, L):
        
        A_prev = A

        A, cache = linear_activation_forward(A_prev, parameters["W"+str(l)], parameters["b"+str(l)], "relu")

        
        D = np.random.rand(*A.shape)            # 1. Generate a dropout mask  
        D = (D < keep_prob).astype(int)         # 2. Convert to binary mask based on the keep_prob
        A = A * D                               # 3. Apply dropout (in which is zero (some neurons))
        A = A/keep_prob                         # Scale activation to maintain expectation         
            
        cache = (cache , D)                     # 4. Caches the dropout mask for use in backpropagation

        caches.append(cache)       


    #Treat the last layer, sigmoid:
    AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], "sigmoid")
    caches.append(cache)

    return AL, caches

In [ ]:
def L_model_backward(AL, Y, caches, dropout_prob=0, layers_drop=[0, 0, 0, 0, 0]):
    grads = {}
    L = len(caches)
    
    
    m = Y.shape[1]
    AL = AL.reshape(Y.shape)

    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    current_cache = caches[L - 1]
    linear_cache, activation_cache = current_cache
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_backward(dAL, linear_cache, "sigmoid")

    for l in reversed(range(L - 1)):
        current_cache = caches[l]
        linear_cache, activation_cache = current_cache

        if layers_drop[l]:                                  # 1. If that was layer dropout was activated
            D = activation_cache[1]                         # 2. Dropout mask
            dA = grads["dA" + str(l + 2)]
            dA = dA * D                                     # 3. Apply mask
            dA = dA / (1 - dropout_prob)                    # 4. Scale again as we did in forward.
            grads["dA" + str(l + 2)] = dA

        grads["dA" + str(l + 1)], grads["dW" + str(l + 1)], grads["db" + str(l + 1)] = linear_backward(grads["dA" + str(l + 2)], linear_cache, "relu")

    return grads